In [60]:
class InfluencerProfile:
    def __init__(self):
        self.appearances = []  # List of video performances
        self.unique_video_ids = set()  # Prevent duplicate video counting
    
    def add_appearance(self, video_performance, video_id):
        if video_id not in self.unique_video_ids:
            self.appearances.append(video_performance)
            self.unique_video_ids.add(video_id)
    
    def get_average_performance(self):
        return sum(self.appearances) / len(self.appearances) if self.appearances else 0

In [61]:
import numpy as np
from deepface import DeepFace

class InfluencerTracker:
    def __init__(self, similarity_threshold=0.8):
        self.influencers = {}  # Key: embedding vector, Value: InfluencerProfile
        self.similarity_threshold = similarity_threshold
    
    def is_similar_face(self, new_embedding, existing_embedding):
        # Compute cosine similarity
        similarity = 1 - np.dot(new_embedding, existing_embedding) / (
            np.linalg.norm(new_embedding) * np.linalg.norm(existing_embedding)
        )
        return similarity < self.similarity_threshold
    
    def add_face(self, face_image, video_performance, video_id):
        # Extract face embedding
        embedding = self.extract_embedding(face_image)
        
        # Check if similar face exists
        for existing_embedding, profile in self.influencers.items():
            if self.is_similar_face(embedding, existing_embedding):
                profile.add_appearance(video_performance, video_id)
                return
        
        # If no similar face, create new profile
        new_profile = InfluencerProfile()
        new_profile.add_appearance(video_performance, video_id)
        self.influencers[embedding] = new_profile
    
    def extract_embedding(self, face_image):
        # Use DeepFace to extract embedding
        embedding = DeepFace.represent(
            img_path=face_image, 
            model_name="Facenet"
        )[0]['embedding']
        return embedding
    
    def get_top_influencers(self, top_n=5):
        # Sort influencers by average performance
        sorted_influencers = sorted(
            self.influencers.values(), 
            key=lambda x: x.get_average_performance(), 
            reverse=True
        )
        return sorted_influencers[:top_n]

In [ ]:
import os
import cv2
import tempfile
from deepface import DeepFace
from urllib.request import urlopen
import shutil
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance



def process_video_frames(frame_directory,j, output_directory="unique_faces", 
                         model_name="Facenet", 
                         distance_metric="cosine", 
                         max_unique_faces = 3,
                         threshold=0.6):
    """
    Process frames from a directory to extract and store unique faces using DeepFace
    
    Args:
        frame_directory (str): Path to directory containing video frames
        output_directory (str): Path to store unique faces
        model_name (str): Face recognition model to use
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold for unique face detection
    """
    # Create directories
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs("temp_unique_faces", exist_ok=True)

    # Track unique faces
    unique_faces_count = 0

    # Iterate through frames
    for frame_filename in sorted(os.listdir(frame_directory)):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frame_directory, frame_filename)
            
            try:
                # Detect faces using DeepFace
                detections = DeepFace.extract_faces(
                    frame_path, 
                    detector_backend="mtcnn", 
                    enforce_detection=True,
                    align=True
                )
                
                print(f"found faces in this frame")
                
                # Process each detected face
                for i, detection in enumerate(detections):
                    if unique_faces_count >= max_unique_faces:
                        return
                    
                    # Extract the face
                    if detection['confidence'] > 0.95:
                        face = detection['face']
                    else:
                        print("low face confidence")
                        continue
                    
                    # Ensure face is valid
                    if face is None or face.size == 0:
                        print(f"Invalid face detected in {frame_filename}")
                        continue
                    
                    # Convert to uint8 if needed
                    if face.dtype != np.uint8:
                        face = (255 * face).astype(np.uint8)
                    
                    # Convert to PIL Image
                    pil_face = Image.fromarray(face)
                    
                    # Enhance face image
                    enhanced_face = enhance_face_image(pil_face)
                    
                    # Temporary path for current face
                    temp_face_path = os.path.join("temp_unique_faces", f"temp_face_{frame_filename}.jpg")
                    enhanced_face.save(temp_face_path)

                    # Check if the image if valid or not i.e it contains a face
                    # try:
                    #     res = DeepFace.extract_faces(img_path=temp_face_path)
                    # except Exception as e:
                    #     continue
                    
                    # Check if the face is unique
                    is_unique = check_unique_face(
                        temp_face_path, 
                        output_directory, 
                        model_name, 
                        distance_metric, 
                        threshold
                    )
                    
                    # Save unique face
                    if is_unique:
                        print("unique face found")
                        unique_filename = f"unique_face_{unique_faces_count}_{j}_{frame_filename}"
                        unique_path = os.path.join(output_directory, unique_filename)
                        print(f"unique_path : {unique_path}")
                        enhanced_face.save(unique_path)
                        unique_faces_count += 1
                        print(f"Saved unique face: {unique_filename}")
                    
                    # Clean up temp face
                    os.remove(temp_face_path)
            
            except Exception as e:
                print(f"Error processing {frame_filename} : No face in this image or frame")

def check_unique_face(face_path, output_directory, model_name, distance_metric, threshold):
    """
    Check if the face is unique compared to existing faces in the output directory
    
    Args:
        face_path (str): Path to the current face image
        output_directory (str): Directory containing existing unique faces
        model_name (str): Face recognition model
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold
    
    Returns:
        bool: True if face is unique, False otherwise
    """
    # If output directory is empty, face is unique
    if not os.listdir(output_directory):
        return True
    
    try:
        # Use DeepFace to find similar faces
        results = DeepFace.find(
            img_path=face_path,
            db_path=output_directory,
            model_name=model_name,
            threshold = 0.8,
            distance_metric=distance_metric
        )
        
        # Check if any similar faces are found
        if results is None or (isinstance(results, pd.DataFrame) and results.empty):
            return True
        
        # If results exist, check the distance
        if isinstance(results, list) and len(results) > 0:
            # Get the minimum distance
            min_distance = results[0]['distance'].min() if not results[0].empty else float('inf')
            return min_distance > threshold
        
        return False
    
    except ValueError as e:
        # If an error occurs (e.g., no faces found), consider it unique
        print(f"Face comparison error: {e}")
        return False

def check_unique_face2(face_path, output_directory, model_name, distance_metric, threshold=0.6):
    """
    Improved unique face checking using multiple verification methods
    """
    # If output directory is empty, face is unique
    existing_faces = os.listdir(output_directory)
    if not existing_faces:
        return True
    
    try:
        # Iterate through existing faces and verify each
        for existing_face in existing_faces:
            existing_face_path = os.path.join(output_directory, existing_face)
            
            try:
                # Use verify method for more precise comparison
                verification_result = DeepFace.verify(
                    img1_path=face_path,
                    img2_path=existing_face_path,                    
                    threshold=0.8  # Adjusted threshold
                )
                
                # If verification returns True (similar face found), it's not unique
                if verification_result['verified']:
                    return False
            
            except Exception as verify_error:
                # If verification fails for a specific image, continue checking others
                print(f"Verification error with {existing_face}: {verify_error}")
                continue
                #return False
        
        # If no similar faces found after checking all existing faces
        return True
    
    except Exception as e:
        print(f"Unique face check error: {e}")
        return False

def enhance_face_image(pil_image, resize_dim=(256, 256)):
    """
    Enhance face image quality
    
    Args:
        pil_image (PIL.Image): Input face image
        resize_dim (tuple): Target resize dimensions
    
    Returns:
        PIL.Image: Enhanced face image
    """
    # Resize image
    resized_image = pil_image.resize(resize_dim, Image.LANCZOS)
    
    # Enhance sharpness
    sharpness_enhancer = ImageEnhance.Sharpness(resized_image)
    sharpened_image = sharpness_enhancer.enhance(2.0)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(sharpened_image)
    enhanced_image = contrast_enhancer.enhance(1.2)
    
    return enhanced_image

# Function to create the main directory for storing unique faces
def create_main_directory(directory="unique_faces"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# Function to create/clear the extracted frames directory
def create_extracted_frames_directory(directory="extracted_frames"):
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Remove existing frames
    os.makedirs(directory)
    return directory

# Function to extract frames from a video
def extract_frames(video_path, frame_count=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)  # Calculate frame interval
    
    for i in range(frame_count):
        frame_index = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

# Function to process videos and extract unique faces
def process_videos(video_urls, performance, main_dir="unique_faces", frame_count=30, model_name="Facenet", distance_metric="cosine", threshold=0.4):
    main_dir = create_main_directory(main_dir)
    
    # Iterate over each video URL
    j = 1
    for video_url in video_urls:
        print(f"Processing video: {video_url}")

        # Create/clear the extracted frames directory for each video
        extracted_frames_dir = create_extracted_frames_directory("extracted_frames")
        
        # Download the video to a temporary file
        with tempfile.NamedTemporaryFile(delete=True, suffix=".mp4") as temp_video:
            temp_video.write(urlopen(video_url).read())
            temp_video.flush()

            # Extract frames from the video
            frames = extract_frames(temp_video.name, frame_count=frame_count)
            
            # Store extracted frames temporarily in the extracted_frames directory
            for frame_index, frame in enumerate(frames):
                frame_path = os.path.join(extracted_frames_dir, f"frame_{frame_index}.jpg")
                cv2.imwrite(frame_path, frame)
            
            # Iterate through extracted frames to detect faces and store unique faces
            process_video_frames(extracted_frames_dir,j)
            
            # Clean up the extracted frames directory after processing
            shutil.rmtree(extracted_frames_dir)
            j += 1
        
        influencer_tracker = InfluencerTracker()
        for face_path in main_dir:
            influencer_tracker.add_face(
                face_image=face_path, 
                video_performance=performance, 
                video_id=video_url
            )
        
        top_influencers = influencer_tracker.get_top_influencers()
        print(top_influencers)

    print("Processing completed.")

# List of video URLs
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399",
    # "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-901695498551491",
    # "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-879631723735807"
]

perfomance = [0.5 , 0.4 , 0.7 , 0.8]



# Run the process
process_videos(video_urls,perfomance)


In [48]:
result = DeepFace.extract_faces(
    img_path = "unique_faces/unique_face_0_1_frame_15.jpg"
)
result

ValueError: Face could not be detected in unique_faces/unique_face_0_1_frame_15.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.

In [66]:
import os
import cv2
import tempfile
from deepface import DeepFace
from urllib.request import urlopen
import shutil
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance

# Existing functions remain the same (process_video_frames, check_unique_face, enhance_face_image, etc.)

class InfluencerProfile:
    def __init__(self):
        self.appearances = []  # List of video performances
        self.unique_video_ids = set()  # Prevent duplicate video counting
        self.face_paths = []  # Store paths of faces for this influencer
    
    def add_appearance(self, video_performance, video_id, face_path):
        if video_id not in self.unique_video_ids:
            self.appearances.append(video_performance)
            self.unique_video_ids.add(video_id)
            self.face_paths.append(face_path)
    
    def get_average_performance(self):
        return sum(self.appearances) / len(self.appearances) if self.appearances else 0

class InfluencerTracker:
    def __init__(self, similarity_threshold=0.8):
        self.influencers = {}  # Key: embedding vector, Value: InfluencerProfile
        self.similarity_threshold = similarity_threshold
    
    def is_similar_face(self, new_embedding, existing_embedding):
        # Compute cosine similarity
        similarity = 1 - np.dot(new_embedding, existing_embedding) / (
            np.linalg.norm(new_embedding) * np.linalg.norm(existing_embedding)
        )
        return similarity < self.similarity_threshold
    
    def add_face(self, face_image, video_performance, video_id):
        # Ensure face_image is a valid file path
        if not os.path.exists(face_image):
            print(f"Warning: File {face_image} does not exist")
            return
        
        try:
            # Extract face embedding
            embedding = self.extract_embedding(face_image)
            
            # Check if similar face exists
            for existing_embedding, profile in self.influencers.items():
                if self.is_similar_face(embedding, existing_embedding):
                    profile.add_appearance(video_performance, video_id, face_image)
                    return
            
            # If no similar face, create new profile
            new_profile = InfluencerProfile()
            new_profile.add_appearance(video_performance, video_id, face_image)
            self.influencers[tuple(embedding)] = new_profile
        
        except Exception as e:
            print(f"Error processing {face_image}: {e}")
    
    def extract_embedding(self, face_image):
        # Verify the image can be processed
        try:
            # Use DeepFace to extract embedding
            embeddings = DeepFace.represent(
                img_path=face_image, 
                model_name="Facenet"
            )
            return embeddings[0]['embedding']
        except Exception as e:
            print(f"Embedding extraction error for {face_image}: {e}")
            raise
    
    def get_top_influencers(self, top_n=5):
        # Sort influencers by average performance
        sorted_influencers = sorted(
            self.influencers.values(), 
            key=lambda x: x.get_average_performance(), 
            reverse=True
        )
        return sorted_influencers[:top_n]

In [69]:
import os
import cv2
import tempfile
from deepface import DeepFace
from urllib.request import urlopen
import shutil
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance



def process_video_frames(frame_directory,j, output_directory="unique_faces", 
                         model_name="Facenet", 
                         distance_metric="cosine", 
                         max_unique_faces = 3,
                         threshold=0.6):
    """
    Process frames from a directory to extract and store unique faces using DeepFace
    
    Args:
        frame_directory (str): Path to directory containing video frames
        output_directory (str): Path to store unique faces
        model_name (str): Face recognition model to use
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold for unique face detection
    """
    # Create directories
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs("temp_unique_faces", exist_ok=True)

    # Track unique faces
    unique_faces_count = 0

    # Iterate through frames
    for frame_filename in sorted(os.listdir(frame_directory)):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frame_directory, frame_filename)
            
            try:
                # Detect faces using DeepFace
                detections = DeepFace.extract_faces(
                    frame_path, 
                    detector_backend="mtcnn", 
                    enforce_detection=True,
                    align=True
                )
                
                print(f"found faces in this frame")
                
                # Process each detected face
                for i, detection in enumerate(detections):
                    if unique_faces_count >= max_unique_faces:
                        return
                    
                    # Extract the face
                    if detection['confidence'] > 0.95:
                        face = detection['face']
                    else:
                        print("low face confidence")
                        continue
                    
                    # Ensure face is valid
                    if face is None or face.size == 0:
                        print(f"Invalid face detected in {frame_filename}")
                        continue
                    
                    # Convert to uint8 if needed
                    if face.dtype != np.uint8:
                        face = (255 * face).astype(np.uint8)
                    
                    # Convert to PIL Image
                    pil_face = Image.fromarray(face)
                    
                    # Enhance face image
                    enhanced_face = enhance_face_image(pil_face)
                    
                    # Temporary path for current face
                    temp_face_path = os.path.join("temp_unique_faces", f"temp_face_{frame_filename}.jpg")
                    enhanced_face.save(temp_face_path)

                    # Check if the image if valid or not i.e it contains a face
                    # try:
                    #     res = DeepFace.extract_faces(img_path=temp_face_path)
                    # except Exception as e:
                    #     continue
                    
                    # Check if the face is unique
                    is_unique = check_unique_face(
                        temp_face_path, 
                        output_directory, 
                        model_name, 
                        distance_metric, 
                        threshold
                    )
                    
                    # Save unique face
                    if is_unique:
                        print("unique face found")
                        unique_filename = f"unique_face_{unique_faces_count}_{j}_{frame_filename}"
                        unique_path = os.path.join(output_directory, unique_filename)
                        print(f"unique_path : {unique_path}")
                        enhanced_face.save(unique_path)
                        unique_faces_count += 1
                        print(f"Saved unique face: {unique_filename}")
                    
                    # Clean up temp face
                    os.remove(temp_face_path)
            
            except Exception as e:
                print(f"Error processing {frame_filename} : No face in this image or frame")

def check_unique_face(face_path, output_directory, model_name, distance_metric, threshold):
    """
    Check if the face is unique compared to existing faces in the output directory
    
    Args:
        face_path (str): Path to the current face image
        output_directory (str): Directory containing existing unique faces
        model_name (str): Face recognition model
        distance_metric (str): Metric for face comparison
        threshold (float): Similarity threshold
    
    Returns:
        bool: True if face is unique, False otherwise
    """
    # If output directory is empty, face is unique
    if not os.listdir(output_directory):
        return True
    
    try:
        # Use DeepFace to find similar faces
        results = DeepFace.find(
            img_path=face_path,
            db_path=output_directory,
            model_name=model_name,
            threshold = 0.8,
            distance_metric=distance_metric
        )
        
        # Check if any similar faces are found
        if results is None or (isinstance(results, pd.DataFrame) and results.empty):
            return True
        
        # If results exist, check the distance
        if isinstance(results, list) and len(results) > 0:
            # Get the minimum distance
            min_distance = results[0]['distance'].min() if not results[0].empty else float('inf')
            return min_distance > threshold
        
        return False
    
    except ValueError as e:
        # If an error occurs (e.g., no faces found), consider it unique
        print(f"Face comparison error: {e}")
        return False

def check_unique_face2(face_path, output_directory, model_name, distance_metric, threshold=0.6):
    """
    Improved unique face checking using multiple verification methods
    """
    # If output directory is empty, face is unique
    existing_faces = os.listdir(output_directory)
    if not existing_faces:
        return True
    
    try:
        # Iterate through existing faces and verify each
        for existing_face in existing_faces:
            existing_face_path = os.path.join(output_directory, existing_face)
            
            try:
                # Use verify method for more precise comparison
                verification_result = DeepFace.verify(
                    img1_path=face_path,
                    img2_path=existing_face_path,                    
                    threshold=0.8  # Adjusted threshold
                )
                
                # If verification returns True (similar face found), it's not unique
                if verification_result['verified']:
                    return False
            
            except Exception as verify_error:
                # If verification fails for a specific image, continue checking others
                print(f"Verification error with {existing_face}: {verify_error}")
                continue
                #return False
        
        # If no similar faces found after checking all existing faces
        return True
    
    except Exception as e:
        print(f"Unique face check error: {e}")
        return False

def enhance_face_image(pil_image, resize_dim=(256, 256)):
    """
    Enhance face image quality
    
    Args:
        pil_image (PIL.Image): Input face image
        resize_dim (tuple): Target resize dimensions
    
    Returns:
        PIL.Image: Enhanced face image
    """
    # Resize image
    resized_image = pil_image.resize(resize_dim, Image.LANCZOS)
    
    # Enhance sharpness
    sharpness_enhancer = ImageEnhance.Sharpness(resized_image)
    sharpened_image = sharpness_enhancer.enhance(2.0)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(sharpened_image)
    enhanced_image = contrast_enhancer.enhance(1.2)
    
    return enhanced_image

# Function to create the main directory for storing unique faces
def create_main_directory(directory="unique_faces"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# Function to create/clear the extracted frames directory
def create_extracted_frames_directory(directory="extracted_frames"):
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Remove existing frames
    os.makedirs(directory)
    return directory

# Function to extract frames from a video
def extract_frames(video_path, frame_count=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)  # Calculate frame interval
    
    for i in range(frame_count):
        frame_index = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames


def process_videos(video_urls, performance, main_dir="unique_faces", frame_count=30, model_name="Facenet", distance_metric="cosine", threshold=0.4):
    # Validate input
    if len(video_urls) != len(performance):
        raise ValueError("Number of video URLs must match number of performance metrics")
    
    # Create main directory for unique faces
    main_dir = create_main_directory(main_dir)
    
    # Initialize influencer tracker
    influencer_tracker = InfluencerTracker()
    
    # Iterate over each video URL
    for j, (video_url, video_performance) in enumerate(zip(video_urls, performance), 1):
        print(f"Processing video: {video_url}")

        # Create/clear the extracted frames directory for each video
        extracted_frames_dir = create_extracted_frames_directory("extracted_frames")
        
        # Download the video to a temporary file
        with tempfile.NamedTemporaryFile(delete=True, suffix=".mp4") as temp_video:
            temp_video.write(urlopen(video_url).read())
            temp_video.flush()

            # Extract frames from the video
            frames = extract_frames(temp_video.name, frame_count=frame_count)
            
            # Store extracted frames temporarily in the extracted_frames directory
            for frame_index, frame in enumerate(frames):
                frame_path = os.path.join(extracted_frames_dir, f"frame_{frame_index}.jpg")
                cv2.imwrite(frame_path, frame)
            
            # Iterate through extracted frames to detect faces and store unique faces
            process_video_frames(extracted_frames_dir, j)
            
            # Clean up the extracted frames directory after processing
            shutil.rmtree(extracted_frames_dir)
    
    # Process unique faces for influencer tracking
    image_files = [
        os.path.join(main_dir, f) 
        for f in os.listdir(main_dir) 
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]
    
    # Add faces to influencer tracker
    for face_path in image_files:
        # Match performance to video URL (you might need to adjust this logic)
        corresponding_video_url = video_urls[image_files.index(face_path) % len(video_urls)]
        corresponding_performance = performance[image_files.index(face_path) % len(performance)]
        
        influencer_tracker.add_face(
            face_image=face_path, 
            video_performance=corresponding_performance, 
            video_id=corresponding_video_url
        )
    
    # Get and print top performing influencers
    top_influencers = influencer_tracker.get_top_influencers()
    
    # Print detailed results
    for i, influencer in enumerate(top_influencers, 1):
        print(f"\nTop Influencer {i}:")
        print(f"  Average Performance: {influencer.get_average_performance():.2f}")
        print(f"  Total Appearances: {len(influencer.appearances)}")
        print(f"  Unique Videos: {len(influencer.unique_video_ids)}")
        print("  Face Paths:")
        for face_path in influencer.face_paths:
            print(f"    - {face_path}")
    
    print(top_influencers)
    return top_influencers

# List of video URLs
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399",
]

performance = [0.5, 0.4, 0.7, 0.8]

# Run the process
process_videos(video_urls, performance)

Processing video: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494
1/1 [==============================] - 0s 18ms/step
found faces in this frame
low face confidence
1/1 [==============================] - 0s 14ms/step
found faces in this frame
unique face found
unique_path : unique_faces/unique_face_0_1_frame_1.jpg
Saved unique face: unique_face_0_1_frame_1.jpg
1/1 [==============================] - 0s 15ms/step
found faces in this frame
24-11-30 20:52:17 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00, 107.26it/s]

24-11-30 20:52:17 - 🔴 Exception while extracting faces from unique_faces/unique_face_0_1_frame_1.jpg: Face could not be detected in unique_faces/unique_face_0_1_frame_1.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
24-11-30 20:52:17 - There are now 1 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-11-30 20:52:17 - Searching temp_unique_faces/temp_face_frame_10.jpg.jpg in 1 length datastore
24-11-30 20:52:17 - find function duration 0.1836380958557129 seconds
unique face found
unique_path : unique_faces/unique_face_1_1_frame_10.jpg
Saved unique face: unique_face_1_1_frame_10.jpg
low face confidence


1/1 [==============================] - 0s 12ms/step
found faces in this frame
24-11-30 20:52:18 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  8.42it/s]

...

24-11-30 20:54:12 - There are now 8 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-11-30 20:54:12 - Searching temp_unique_faces/temp_face_frame_17.jpg.jpg in 8 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_17.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence


1/1 [==============================] - 0s 16ms/step
found faces in this frame
24-11-30 20:54:13 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 8 length datastore
24-11-30 20:54:13 - find function duration 0.12437200546264648 seconds
unique face found
unique_path : unique_faces/unique_face_1_4_frame_18.jpg
Saved unique face: unique_face_1_4_frame_18.jpg
24-11-30 20:54:13 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  8.63it/s]

24-11-30 20:54:13 - There are now 9 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-11-30 20:54:13 - Searching temp_unique_faces/temp_face_frame_18.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_18.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


1/1 [==============================] - 0s 15ms/step
found faces in this frame
24-11-30 20:54:14 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 9 length datastore
24-11-30 20:54:14 - find function duration 0.11467194557189941 seconds
24-11-30 20:54:14 - Searching temp_unique_faces/temp_face_frame_19.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_19.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence
1/1 [==============================] - 0s 14ms/step
found faces in this frame
24-11-30 20:54:15 - Searching temp_unique_faces/temp_face_frame_2.jpg.jpg in 9 length datastore
Face comparison error: Face could not be detected in temp_unique_faces/temp_face_frame_2.jpg.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
low face confidence
1/1 [==============================] - 0s 16ms

In [70]:
import os
import numpy as np
import pandas as pd
import cv2
import tempfile
from urllib.request import urlopen
import shutil
from deepface import DeepFace
from PIL import Image, ImageEnhance
import faiss

class InfluencerTracker:
    def __init__(self, embedding_model='Facenet', dimension=512):
        """
        Initialize the Influencer Tracker with embedding storage
        
        Args:
            embedding_model (str): Face embedding model to use
            dimension (int): Dimensionality of face embeddings
        """
        # Initialize FAISS index for efficient similarity search
        self.index = faiss.IndexFlatL2(dimension)
        
        # Store metadata
        self.metadata = {
            'face_paths': [],
            'performances': [],
            'video_ids': []
        }
        
        # Embedding model
        self.embedding_model = embedding_model
    
    def compute_embedding(self, image_path):
        """
        Compute face embedding for a given image
        
        Args:
            image_path (str): Path to the image
        
        Returns:
            numpy.ndarray: Face embedding vector
        """
        try:
            # Compute embedding using DeepFace
            embedding = DeepFace.represent(
                img_path=image_path, 
                model_name=self.embedding_model,
                enforce_detection=True
            )[0]['embedding']
            
            return np.array(embedding).astype('float32')
        
        except Exception as e:
            print(f"Error computing embedding for {image_path}: {e}")
            return None
    
    def add_face(self, face_image, video_performance, video_id):
        """
        Add a unique face to the tracker
        
        Args:
            face_image (str): Path to face image
            video_performance (float): Performance metric
            video_id (str): Unique video identifier
        
        Returns:
            bool: Whether face was successfully added
        """
        # Compute embedding
        embedding = self.compute_embedding(face_image)
        
        if embedding is None:
            return False
        
        # Check for similar faces
        similar_index = self.find_similar_face(embedding)
        
        if similar_index is not None:
            # Update performance of existing face
            self.metadata['performances'][similar_index] = np.mean([
                self.metadata['performances'][similar_index], 
                video_performance
            ])
            return False
        
        # Add new face
        self.index.add(embedding.reshape(1, -1))
        self.metadata['face_paths'].append(face_image)
        self.metadata['performances'].append(video_performance)
        self.metadata['video_ids'].append(video_id)
        
        return True
    
    def find_similar_face(self, embedding, threshold=0.7):
        """
        Find similar face in the index
        
        Args:
            embedding (numpy.ndarray): Face embedding to match
            threshold (float): Similarity threshold
        
        Returns:
            int or None: Index of similar face, or None
        """
        if self.index.ntotal == 0:
            return None
        
        # Search for nearest neighbors
        D, I = self.index.search(embedding.reshape(1, -1), 1)
        
        # If distance is below threshold, return index
        if D[0][0] < threshold:
            return I[0][0]
        
        return None
    
    def get_top_influencers(self, top_n=5):
        """
        Get top performing influencers
        
        Args:
            top_n (int): Number of top influencers to return
        
        Returns:
            pandas.DataFrame: Top influencers with their details
        """
        # Create DataFrame
        df = pd.DataFrame({
            'face_path': self.metadata['face_paths'],
            'performance': self.metadata['performances'],
            'video_id': self.metadata['video_ids']
        })
        
        # Group by face path and compute average performance
        grouped = df.groupby('face_path').agg({
            'performance': 'mean',
            'video_id': 'nunique'
        }).reset_index()
        
        # Sort by performance
        top_influencers = grouped.sort_values('performance', ascending=False).head(top_n)
        
        return top_influencers

def process_videos(video_urls, performance, main_dir="unique_faces", frame_count=30):
    """
    Process multiple videos to track unique influencers
    
    Args:
        video_urls (list): List of video URLs
        performance (list): Performance metrics for videos
        main_dir (str): Directory to store unique faces
        frame_count (int): Number of frames to extract per video
    
    Returns:
        pandas.DataFrame: Top performing influencers
    """
    # Create main directory
    os.makedirs(main_dir, exist_ok=True)
    
    # Initialize influencer tracker
    tracker = InfluencerTracker()
    
    # Process each video
    for j, (video_url, video_performance) in enumerate(zip(video_urls, performance), 1):
        print(f"Processing video: {video_url}")
        
        # Create temporary extracted frames directory
        with tempfile.TemporaryDirectory() as extracted_frames_dir:
            # Download video
            with tempfile.NamedTemporaryFile(delete=True, suffix=".mp4") as temp_video:
                temp_video.write(urlopen(video_url).read())
                temp_video.flush()
                
                # Extract frames
                frames = extract_frames(temp_video.name, frame_count)
                
                # Save frames
                for frame_index, frame in enumerate(frames):
                    frame_path = os.path.join(extracted_frames_dir, f"frame_{frame_index}.jpg")
                    cv2.imwrite(frame_path, frame)
                
                # Detect and process unique faces
                unique_faces = detect_unique_faces(extracted_frames_dir, main_dir)
                
                # Add unique faces to tracker
                for face_path in unique_faces:
                    tracker.add_face(
                        face_image=face_path, 
                        video_performance=video_performance, 
                        video_id=video_url
                    )
    
    # Get top influencers
    top_influencers = tracker.get_top_influencers()
    
    # Print results
    print("\nTop Influencers:")
    print(top_influencers)
    
    return top_influencers

def extract_frames(video_path, frame_count=30):
    """
    Extract frames from a video
    
    Args:
        video_path (str): Path to video file
        frame_count (int): Number of frames to extract
    
    Returns:
        list: Extracted frames
    """
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // frame_count, 1)
    
    for i in range(frame_count):
        frame_index = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

def detect_unique_faces(frame_directory, output_directory, max_unique_faces=3):
    """
    Detect unique faces in frames
    
    Args:
        frame_directory (str): Directory containing frames
        output_directory (str): Directory to save unique faces
        max_unique_faces (int): Maximum number of unique faces to save per video
    
    Returns:
        list: Paths of unique faces detected
    """
    unique_faces = []
    
    for frame_filename in sorted(os.listdir(frame_directory)):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frame_directory, frame_filename)
            
            try:
                # Detect faces
                detections = DeepFace.extract_faces(
                    frame_path, 
                    detector_backend="mtcnn", 
                    enforce_detection=True,
                    align=True
                )
                
                # Process each detected face
                for detection in detections:
                    if len(unique_faces) >= max_unique_faces:
                        break
                    
                    # Extract high confidence faces
                    if detection['confidence'] > 0.95:
                        face = detection['face']
                    else:
                        continue
                    
                    # Convert to PIL Image and enhance
                    pil_face = Image.fromarray(face)
                    enhanced_face = enhance_face_image(pil_face)
                    
                    # Save unique face
                    unique_filename = f"unique_face_{len(unique_faces)}_{frame_filename}"
                    unique_path = os.path.join(output_directory, unique_filename)
                    enhanced_face.save(unique_path)
                    
                    unique_faces.append(unique_path)
            
            except Exception as e:
                print(f"Error processing {frame_filename}: {e}")
    
    return unique_faces

def enhance_face_image(pil_image, resize_dim=(256, 256)):
    """
    Enhance face image quality
    
    Args:
        pil_image (PIL.Image): Input face image
        resize_dim (tuple): Target resize dimensions
    
    Returns:
        PIL.Image: Enhanced face image
    """
    # Resize image
    resized_image = pil_image.resize(resize_dim, Image.LANCZOS)
    
    # Enhance sharpness
    sharpness_enhancer = ImageEnhance.Sharpness(resized_image)
    sharpened_image = sharpness_enhancer.enhance(2.0)
    
    # Enhance contrast
    contrast_enhancer = ImageEnhance.Contrast(sharpened_image)
    enhanced_image = contrast_enhancer.enhance(1.2)
    
    return enhanced_image

# Example usage
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-987643572956494",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405",
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-905739711170399",
]

performance = [0.5, 0.4, 0.7, 0.8]

# Process videos and get top influencers
top_influencers = process_videos(video_urls, performance)

1/1 [==============================] - 0s 15ms/step
Error processing frame_1.jpg: Cannot handle this data type: (1, 1, 3), <f8
1/1 [==============================] - 0s 14ms/step
Error processing frame_10.jpg: Cannot handle this data type: (1, 1, 3), <f8
1/1 [==============================] - 0s 16ms/step
Error processing frame_11.jpg: Face could not be detected in /var/folders/cf/47mgd2gx2r17z1vchly_bhd00000gn/T/tmpyf8oe6e_/frame_11.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1/1 [==============================] - 0s 15ms/step
Error processing frame_13.jpg: Face could not be detected in /var/folders/cf/47mgd2gx2r17z1vchly_bhd00000gn/T/tmpyf8oe6e_/frame_13.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1/1 [==============================] - 0s 11ms/step


KeyboardInterrupt: 

In [98]:
dfs = DeepFace.find(
    img_path = "unique_faces/unique_face_2_1_frame_26.jpg",
    db_path = "unique_faces",
    threshold = 0.8
   
)
dfs[0]

24-12-01 00:46:48 - Searching unique_faces/unique_face_2_1_frame_26.jpg in 9 length datastore
24-12-01 00:46:48 - find function duration 0.3922438621520996 seconds


,identity,hash,target_x,target_y,target_w,target_h,source_x,source_y,source_w,source_h,threshold,distance
0,unique_faces/unique_face_2_1_frame_26.jpg,710fb03d1ba964de9c2e894f4afb60402fc6f139,0,0,255,255,0,0,255,255,0.8,0.000000
1,unique_faces/unique_face_1_2_frame_4.jpg,fea7e8b6f7b79edef1df50e8534e88c1a88591ec,30,7,225,248,0,0,255,255,0.8,0.632853
2,unique_faces/unique_face_0_4_frame_16.jpg,3d2b734d55179dd8222cb43fe1551be0c1cafc75,0,0,255,255,0,0,255,255,0.8,0.693293
3,unique_faces/unique_face_0_2_frame_18.jpg,23f1c16bec9f32309dfb868979ca56ed9b9e142e,0,43,226,212,0,0,255,255,0.8,0.735556


In [103]:
dfs = DeepFace.find(
    img_path = "extracted_frames-test/frame_19.jpg",
    db_path = "unique_faces",
    threshold = 0.6
)

dfs

24-12-01 00:51:37 - Searching extracted_frames-test/frame_19.jpg in 9 length datastore


ValueError: Face could not be detected in extracted_frames-test/frame_19.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.

In [104]:
columns = ['image_path', 'hash', 'score', 'no_of_occurance']

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

In [ ]:
def handle_unique(output_directory , face_path , df , model_name , distance_metric , score):
    hash = DeepFace.find(
            img_path=face_path,
            db_path=output_directory,
            model_name=model_name,
            threshold = 0.8,
            distance_metric=distance_metric
        )[0]['hash']
    
    new_row = {
        'image_path' : face_path,
        'hash' : hash,
        'score' : score,
        'no_of_occurance' : 1
    }

    df.append(new_row)


In [ ]:
def handle_not_unique(results , df , score):
    hash = results[0]['hash']

    # Update the 'Age' of the row where 'Name' is 'Bob'
    df.loc[df['hash'] == 'hash', 'score'] += score
    df.loc[df['hash'] == 'hash', 'no_of_occurance'] += 1
    

In [107]:
columns = ['image_path', 'hash', 'score', 'no_of_occurance']

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

new_row = {
        'image_path' : "hello",
        'hash' : 2,
        'score' : 2,
        'no_of_occurance' : 1
    }

df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
df

,image_path,hash,score,no_of_occurance
0,hello,2,2,1


In [108]:
df

,image_path,hash,score,no_of_occurance
0,hello,2,2,1


In [109]:
score = 3
hash = 2
df.loc[df['hash'] == hash, 'score'] += score
df

,image_path,hash,score,no_of_occurance
0,hello,2,5,1


In [119]:
dfs = DeepFace.find(
    img_path = "unique_faces/unique_face_2_1_frame_26.jpg",
    db_path = "unique_faces",
    threshold = 0.7
)



24-12-01 14:03:14 - Searching unique_faces/unique_face_2_1_frame_26.jpg in 9 length datastore
24-12-01 14:03:14 - find function duration 0.5036149024963379 seconds


In [123]:
dfs[0].iloc[0]['hash']

'8226c234a5b191ac178fa2bd03b058a2dcd53fc5'

In [125]:
df

,image_path,hash,score,no_of_occurance
0,hello,2,5,1


In [128]:
def update():
    
    global df
    score = 3
    hash = 2
    df.loc[df['hash'] == hash, 'score'] += score

update()
df

,image_path,hash,score,no_of_occurance
0,hello,2,14,1


In [129]:
df

,image_path,hash,score,no_of_occurance
0,hello,2,14,1


In [133]:
if df.loc[df['hash'] == 2, 'score'].iloc[0] == 14:
    print("true haha")
else:
    print("no")

true haha


In [2]:
import os
os.chdir("/Users/vanshbansal/Desktop/FuelGrowth")
%pwd

'/Users/vanshbansal/Desktop/FuelGrowth'

In [6]:
import pandas as pd
from IPython.display import Image, HTML

# Sample DataFrame with image paths
df = pd.read_excel("temp_data/model_training_data/2024-12-01_19-26-30.xlsx")



# Function to display images in a DataFrame cell
def path_to_image_html(path):
    if os.path.exists(path):  # Check if the file exists
        return f'<img src="{path}" width="100" height="100"/>'
    else:
        return '<p>Image not found</p>'  # Fallback if the image path is invalid

# Apply the function to the DataFrame column
df['image'] = df['image_path'].apply(path_to_image_html)

# Display the DataFrame with HTML-rendered images
HTML(df.to_html(escape=False))


,Unnamed: 0,image_path,hash,score,no_of_occurance,recent_occurance,image
0,0,unique_faces/unique_face_1_1_frame_10.jpg,68b3892fd77c8b54b4e5ba82ebccf580f82d60d8,0.7,2,2,
1,1,unique_faces/unique_face_2_1_frame_26.jpg,cafa327549c3fbfa00a88372a93efe5e691de732,0.7,2,2,
2,2,unique_faces/unique_face_0_2_frame_18.jpg,69f198594b92a7114ef63582df0ab154ff4a0edc,0.3,1,2,
3,3,unique_faces/unique_face_1_2_frame_4.jpg,98308f47bbea8c8d0b2346aaf60b755e3420d3f8,0.3,1,2,


In [11]:

import pandas as pd
from IPython.display import Image, HTML

# Sample DataFrame with image paths
df = pd.read_excel("temp_data/model_training_data/2024-12-01_19-26-30.xlsx")


# Function to render images in HTML
def display_images_in_df(df):
    html_string = "<table>"
    
    # Loop through the DataFrame rows and add images
    for _, row in df.iterrows():
        img_path = row['image_path']
        if os.path.exists(img_path):  # Ensure the image exists
            html_string += f'<tr><td><img src="{img_path}" width="100" height="100"/></td></tr>'
            print(html_string)
        else:
            html_string += f'<tr><td>Image not found for {img_path}</td></tr>'
    
    html_string += "</table>"
    
    # Display the HTML content
    display(HTML(html_string))

# Call the function to display images in the DataFrame
display_images_in_df(df)

<table><tr><td><img src="unique_faces/unique_face_1_1_frame_10.jpg" width="100" height="100"/></td></tr>
<table><tr><td><img src="unique_faces/unique_face_1_1_frame_10.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_2_1_frame_26.jpg" width="100" height="100"/></td></tr>
<table><tr><td><img src="unique_faces/unique_face_1_1_frame_10.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_2_1_frame_26.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_0_2_frame_18.jpg" width="100" height="100"/></td></tr>
<table><tr><td><img src="unique_faces/unique_face_1_1_frame_10.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_2_1_frame_26.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_0_2_frame_18.jpg" width="100" height="100"/></td></tr><tr><td><img src="unique_faces/unique_face_1_2_frame_4.jpg" width="100" height="100"/></td></tr>

""
""
""
""


In [ ]:
from IPython.display import Image, display

# Test a few image paths directly
test_paths = ['unique_faces/unique_face_1_1_frame_10.jpg', 'unique_faces/unique_face_2_1_frame_26.jpg', 'unique_faces/unique_face_1_2_frame_4.jpg']

for path in test_paths:
    if os.path.exists(path):
        display(Image(filename=path))
    else:
        print(f"Image not found: {path}")


In [ ]:
import os
import pandas as pd
from IPython.display import Image, display, HTML

# Sample DataFrame with image paths
df = pd.read_excel("temp_data/model_training_data/2024-12-01_19-26-30.xlsx")

# Function to display images in DataFrame
def display_images_in_df(df):
    
    for _, row in df.iterrows():
        img_path = row['image_path']
        
        # Debugging: print the image path
        print(f"Checking image: {img_path}")
        
        if os.path.exists(img_path):  # Check if image exists
            display(Image(filename=img_path))
        else:
            print("error")
    
    

# Call the function to display images
display_images_in_df(df)


In [28]:
import os
import pandas as pd
from IPython.display import HTML

# Load your DataFrame from Excel
df = pd.read_excel("temp_data/model_training_data/2024-12-01_19-26-30.xlsx")

# Function to convert image paths to HTML img tags
def path_to_image_html(path):
    # Check if the file exists
    if os.path.exists(path):
        return f'<img src="{path}" width="100" height="100"/>'
    else:
        return 'Image not found'  # If the image doesn't exist, return a placeholder message

# Apply the function to create a new column with HTML image tags
df['image'] = df['image_path'].apply(path_to_image_html)

# Display the DataFrame with images rendered as HTML
HTML(df.to_html(escape=False))


,Unnamed: 0,image_path,hash,score,no_of_occurance,recent_occurance,image
0,0,unique_faces/unique_face_1_1_frame_10.jpg,68b3892fd77c8b54b4e5ba82ebccf580f82d60d8,0.7,2,2,
1,1,unique_faces/unique_face_2_1_frame_26.jpg,cafa327549c3fbfa00a88372a93efe5e691de732,0.7,2,2,
2,2,unique_faces/unique_face_0_2_frame_18.jpg,69f198594b92a7114ef63582df0ab154ff4a0edc,0.3,1,2,
3,3,unique_faces/unique_face_1_2_frame_4.jpg,98308f47bbea8c8d0b2346aaf60b755e3420d3f8,0.3,1,2,


In [29]:
df.to_excel("test2.xlsx")

In [3]:
import os
os.chdir("/Users/vanshbansal/Desktop/FuelGrowth")
%pwd

'/Users/vanshbansal/Desktop/FuelGrowth'

In [7]:
import pandas as pd
df = pd.read_excel("temp_data/url_score data/clean/2024-12-01_18-26-15.xlsx")

video_urls = df['url'].to_list()
video_urls

['https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-891391212618023',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904174908300812',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-911172609864769',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-813828049840959',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-938232524809973',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-811606650922029',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-880568190186256',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-611188517518682',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-902456848100089',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-621652523105267',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-951885082053340',
 'https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-9899303031